In [1]:
import sys
import os

if 'google.colab' in sys.modules:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
      drive.mount('/content/drive/')
    %cd drive/MyDrive/nov23_bds_rakuten
else:
    root_files = os.listdir()
    if not "requirements.txt" in root_files:
        print("The jupyter server root should be set to root of git project")
        print("If you're on vscode add the following to your settings.json file")
        print('"jupyter.notebookFileRoot": "${workspaceFolder}"')

Mounted at /content/drive/
/content/drive/MyDrive/nov23_bds_rakuten


In [2]:
random_state = 42
output_dir = "output/streamlit-data-preparation"

In [43]:
import numpy as np
import pandas as pd


from pathlib import Path
import re
from tqdm import tqdm
import pickle as pickle
import zipfile

import pickle

import shutil

In [4]:
def load_pickle(path):
  path_obj = Path(path)
  if path_obj.exists():
    print("Load pickle file from",path)
    with open(path, 'rb') as file:
      return pickle.load(file)
  else:
    print("No pickle file found at",path)
    return None

In [5]:
def get_average_pred(img_pred,text_pred,img_pred_weight=0.5,text_pred_weight=0.5):
  combined_pred = (img_pred * img_pred_weight) + (text_pred * text_pred_weight)
  final_pred = np.argmax(combined_pred, axis=1)
  return final_pred

In [44]:
def import_images(zip_path,dest_dir):
    image_path = dest_dir
    image_dir = Path(image_path)
    if image_dir.exists():
        print("images already imported from drive")
    else:
        print("extracting images from drive into colab's session storage... (about 1 min)")
        with zipfile.ZipFile(zip_path,"r") as images_zip:
            images_zip.extractall(image_path)
    image_path = image_path + "/images"
    return image_path

In [6]:
df_test = pd.read_csv("output/data-preprocessing-resampling/df_test.csv",index_col=0)
df_test.head()

,productid,imageid,text,lang,text_fr,imagefile,prdtypecode
index,,,,,,,
29927,3469563513,1215209457,Adventure Parks Heavy Duty Tree Strap - Dont t...,en,Les parcs d'aventure STRAP DURT DE DURG - N'at...,1280/image_1215209457_product_3469563513.jpg,1280
42596,4152889175,1309607798,Magideal 2 Pcs Orff Instrument De Musique Hoch...,fr,Magideal 2 Pcs Orff Instrument De Musique Hoch...,1320/image_1309607798_product_4152889175.jpg,1320
158,1190507078,1068211845,Casa Padrino Baroque Dining Room With Armrests...,en,Casa Padrino Baroque Dining Room avec des acco...,1560/image_1068211845_product_1190507078.jpg,1560
20061,4236041996,1324088536,T6 Xml Zoomable Led Lampe De Poche Tactique + ...,fr,T6 Xml Zoomable Led Lampe De Poche Tactique + ...,1302/image_1324088536_product_4236041996.jpg,1302
20931,3836739131,1250952042,Organisateur Enfants En Bois Keepsake Cadeau P...,fr,Organisateur Enfants En Bois Keepsake Cadeau P...,1280/image_1250952042_product_3836739131.jpg,1280


In [7]:
classes_ordered = list(np.sort(np.array(df_test["prdtypecode"].unique(),dtype="str")).astype("int"))

In [9]:
test_pred_prob_image = load_pickle("output/data-modeling-fusion/test_pred_image_431_331.pkl")
test_pred_prob_text = load_pickle("output/data-modeling-fusion/test_pred_text_modele_camembert_028.pkl")

Load pickle file from output/data-modeling-fusion/test_pred_image_431_331.pkl
Load pickle file from output/data-modeling-fusion/test_pred_text_modele_camembert_028.pkl


In [10]:
test_pred_image = np.argmax(test_pred_prob_image, axis=1)
test_pred_text = np.argmax(test_pred_prob_text, axis=1)

In [12]:
test_pred = get_average_pred(test_pred_prob_image,test_pred_prob_text,
                             img_pred_weight=0.5,
                             text_pred_weight=0.5)

array([ 8,  9, 10,  8,  4,  4, 17, 11,  8,  9])

In [21]:
test_pred_image = [classes_ordered[c] for c in test_pred_image]
test_pred_text = [classes_ordered[c] for c in test_pred_text]
test_pred = [classes_ordered[c] for c in test_pred]

In [25]:
prediction_correct_image = df_test["prdtypecode"] == test_pred_image
prediction_correct_text = df_test["prdtypecode"] == test_pred_text
prediction_correct_fusion = df_test["prdtypecode"] == test_pred


In [29]:
df_test["prediction_correct_image"] = prediction_correct_image
df_test["prediction_correct_text"] = prediction_correct_text
df_test["prediction_correct_fusion"] = prediction_correct_fusion

In [31]:
Path(output_dir).mkdir(parents=True, exist_ok=True)
df_test.to_csv(output_dir+"/df_streamlit.csv")

In [34]:
df_streamlit = pd.read_csv(output_dir+"/df_streamlit.csv", index_col=0)
df_streamlit.head()

,productid,imageid,text,lang,text_fr,imagefile,prdtypecode,prediction_correct_image,prediction_correct_text,prediction_correct_fusion
index,,,,,,,,,,
29927,3469563513,1215209457,Adventure Parks Heavy Duty Tree Strap - Dont t...,en,Les parcs d'aventure STRAP DURT DE DURG - N'at...,1280/image_1215209457_product_3469563513.jpg,1280,False,False,False
42596,4152889175,1309607798,Magideal 2 Pcs Orff Instrument De Musique Hoch...,fr,Magideal 2 Pcs Orff Instrument De Musique Hoch...,1320/image_1309607798_product_4152889175.jpg,1320,False,True,True
158,1190507078,1068211845,Casa Padrino Baroque Dining Room With Armrests...,en,Casa Padrino Baroque Dining Room avec des acco...,1560/image_1068211845_product_1190507078.jpg,1560,False,True,True
20061,4236041996,1324088536,T6 Xml Zoomable Led Lampe De Poche Tactique + ...,fr,T6 Xml Zoomable Led Lampe De Poche Tactique + ...,1302/image_1324088536_product_4236041996.jpg,1302,True,True,True
20931,3836739131,1250952042,Organisateur Enfants En Bois Keepsake Cadeau P...,fr,Organisateur Enfants En Bois Keepsake Cadeau P...,1280/image_1250952042_product_3836739131.jpg,1280,False,True,True


In [42]:
df_streamlit_subset = pd.concat([
  df_streamlit[( df_streamlit["prediction_correct_fusion"] \
              &  ~df_streamlit["prediction_correct_image"] & ~df_streamlit["prediction_correct_text"] )].head(5),

  df_streamlit[( df_streamlit["prediction_correct_fusion"] \
              &  df_streamlit["prediction_correct_image"] & ~df_streamlit["prediction_correct_text"] )].head(5),

  df_streamlit[( df_streamlit["prediction_correct_fusion"] \
              &  ~df_streamlit["prediction_correct_image"] & df_streamlit["prediction_correct_text"] )].head(5),

  df_streamlit[( df_streamlit["prediction_correct_fusion"] \
              &  df_streamlit["prediction_correct_image"] & df_streamlit["prediction_correct_text"] )].head(5)
],axis=0)

df_streamlit_subset

,productid,imageid,text,lang,text_fr,imagefile,prdtypecode,prediction_correct_image,prediction_correct_text,prediction_correct_fusion
index,,,,,,,,,,
39243,1983972046,1118654744,Ambulance Retrofriction Son Jouet - (1,en,Ambulance Retrofriction Son Jouet - (1,1280/image_1118654744_product_1983972046.jpg,1280,False,False,True
45694,154099350,909766184,Jeux Olympiques Atlanta 1996 The Atlanta Journ...,fr,Jeux Olympiques Atlanta 1996 The Atlanta Journ...,2403/image_909766184_product_154099350.jpg,2403,False,False,True
19766,3971761681,1270628131,Lampe Led Étanche Mur Solaire Avec Double Coul...,fr,Lampe Led Étanche Mur Solaire Avec Double Coul...,2060/image_1270628131_product_3971761681.jpg,2060,False,False,True
1543,3847027140,1252097056,Lampe Led Humidificateur Coupe Lumière Mignon ...,fr,Lampe Led Humidificateur Coupe Lumière Mignon ...,2060/image_1252097056_product_3847027140.jpg,2060,False,False,True
67376,3032797049,1186580470,U Draw Studio Et Tablette -,en,U dessiner studio et tablette -,2462/image_1186580470_product_3032797049.jpg,2462,True,False,True
76736,3590004082,1225434717,Set De 5 Tables Gigognes - H des 4 petites tab...,fr,Set De 5 Tables Gigognes - H des 4 petites tab...,1560/image_1225434717_product_3590004082.jpg,1560,True,False,True
5598,2063162058,1251713806,L'héritage De Charlotte - Tome 2 -,fr,L'héritage De Charlotte - Tome 2 -,10/image_1251713806_product_2063162058.jpg,10,True,False,True
5786,557984116,1026166369,Angry Birds Lot De 33 Cartes Dont 3 Holos - jo...,fr,Angry Birds Lot De 33 Cartes Dont 3 Holos - jo...,1281/image_1026166369_product_557984116.jpg,1281,True,False,True
9099,3530750,114783730,Celui Qui Tire Dans Le Ciel = Skyshooter -,fr,Celui Qui Tire Dans Le Ciel = Skyshooter -,1160/image_114783730_product_3530750.jpg,1160,True,False,True


In [45]:
image_path = import_images("output/data-preprocessing-images/images_pre.zip","/tmp/rakuten-images-2")

extracting images from drive into colab's session storage... (about 1 min)


In [48]:
tmp_img_path = "/tmp/images_streamlit"
Path(tmp_img_path).mkdir(parents=True, exist_ok=True)
for index,row in tqdm(df_streamlit_subset.iterrows()):
  image_folder = tmp_img_path+"/"+str(row["prdtypecode"])
  Path(image_folder).mkdir(parents=True, exist_ok=True)
  #print("copy",image_path+"/"+row["imagefile"], tmp_img_path+"/"+row["imagefile"])
  shutil.copyfile(image_path+"/"+row["imagefile"], tmp_img_path+"/"+row["imagefile"])

19it [00:00, 556.91it/s]


In [49]:
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        print("zipping ",root,"...")
        for file in tqdm(files):
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))

with zipfile.ZipFile('/tmp/images_streamlit.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir(tmp_img_path, zipf)

zipping  /tmp/images_streamlit ...


0it [00:00, ?it/s]


zipping  /tmp/images_streamlit/1300 ...


100%|██████████| 1/1 [00:00<00:00, 354.61it/s]


zipping  /tmp/images_streamlit/1302 ...


...

100%|██████████| 1/1 [00:00<00:00, 133.04it/s]


zipping  /tmp/images_streamlit/1280 ...


100%|██████████| 2/2 [00:00<00:00, 303.31it/s]


zipping  /tmp/images_streamlit/1320 ...


100%|██████████| 2/2 [00:00<00:00, 764.13it/s]


In [50]:
shutil.copyfile("/tmp/images_streamlit.zip", output_dir+"/images_streamlit.zip")

'output/streamlit-data-preparation/images_streamlit.zip'

In [51]:
df_streamlit_subset.to_csv(output_dir+"/df_streamlit_subset.csv")

In [52]:
pd.read_csv(output_dir+"/df_streamlit_subset.csv",index_col=0)

,productid,imageid,text,lang,text_fr,imagefile,prdtypecode,prediction_correct_image,prediction_correct_text,prediction_correct_fusion
index,,,,,,,,,,
39243,1983972046,1118654744,Ambulance Retrofriction Son Jouet - (1,en,Ambulance Retrofriction Son Jouet - (1,1280/image_1118654744_product_1983972046.jpg,1280,False,False,True
45694,154099350,909766184,Jeux Olympiques Atlanta 1996 The Atlanta Journ...,fr,Jeux Olympiques Atlanta 1996 The Atlanta Journ...,2403/image_909766184_product_154099350.jpg,2403,False,False,True
19766,3971761681,1270628131,Lampe Led Étanche Mur Solaire Avec Double Coul...,fr,Lampe Led Étanche Mur Solaire Avec Double Coul...,2060/image_1270628131_product_3971761681.jpg,2060,False,False,True
1543,3847027140,1252097056,Lampe Led Humidificateur Coupe Lumière Mignon ...,fr,Lampe Led Humidificateur Coupe Lumière Mignon ...,2060/image_1252097056_product_3847027140.jpg,2060,False,False,True
67376,3032797049,1186580470,U Draw Studio Et Tablette -,en,U dessiner studio et tablette -,2462/image_1186580470_product_3032797049.jpg,2462,True,False,True
76736,3590004082,1225434717,Set De 5 Tables Gigognes - H des 4 petites tab...,fr,Set De 5 Tables Gigognes - H des 4 petites tab...,1560/image_1225434717_product_3590004082.jpg,1560,True,False,True
5598,2063162058,1251713806,L'héritage De Charlotte - Tome 2 -,fr,L'héritage De Charlotte - Tome 2 -,10/image_1251713806_product_2063162058.jpg,10,True,False,True
5786,557984116,1026166369,Angry Birds Lot De 33 Cartes Dont 3 Holos - jo...,fr,Angry Birds Lot De 33 Cartes Dont 3 Holos - jo...,1281/image_1026166369_product_557984116.jpg,1281,True,False,True
9099,3530750,114783730,Celui Qui Tire Dans Le Ciel = Skyshooter -,fr,Celui Qui Tire Dans Le Ciel = Skyshooter -,1160/image_114783730_product_3530750.jpg,1160,True,False,True
